In [20]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import *

# ---------------------------------------------------------
# 1) Geometrie: Rechteck 0≤x≤28, 0≤y≤1  mit Kreisloch
# ---------------------------------------------------------
rect = Rectangle(Pnt(0,0), Pnt(28,1)).bc("wall","wall","wall","inlet") 
# Reihenfolge: bottom, right, top, left → inlet = links
circle = Circle(Pnt(7,0.5), 0.5).bc("circle")

geo = rect - circle      # Loch ausschneiden

mesh = Mesh(geo.GenerateMesh(maxh=0.15))
mesh.Curve(3)

print(mesh.GetBoundaries())  # Prüfen

# ---------------------------------------------------------
# 2) Taylor–Hood FE-Räume
# ---------------------------------------------------------
V = VectorH1(mesh, order=2, dirichlet="wall|circle|inlet")
Q = H1(mesh, order=1)

X = V * Q
(u, p), (v, q) = X.TnT()

gfu = GridFunction(X)
velocity, pressure = gfu.components

# ---------------------------------------------------------
# 3) Randwerte (Inflow-Profil)
# ---------------------------------------------------------
uin = CoefficientFunction((4*y*(1-y), 0))   # parabolischer Einströmer
velocity.Set(uin, definedon=mesh.Boundaries("inlet"))
velocity.Set((0,0), definedon=mesh.Boundaries("wall|circle"))

# ---------------------------------------------------------
# 4) STOKES-Lösung
# ---------------------------------------------------------
nu = 0.02  # Viskosität

stokes = BilinearForm(X)
stokes += nu*InnerProduct(grad(u), grad(v))*dx
stokes += -div(u)*q*dx - div(v)*p*dx
stokes.Assemble()

rhs = LinearForm(X)
rhs.Assemble()

# Lifting für inhomogene Dirichlet-Bedingungen
res = rhs.vec - stokes.mat * gfu.vec
invst = stokes.mat.Inverse(freedofs=X.FreeDofs(), inverse="umfpack")
gfu.vec.data += invst * res

Draw(velocity, mesh, "stokes-velocity")

# ---------------------------------------------------------
# 5) NAVIER–STOKES durch Zeitintegration
# ---------------------------------------------------------
dt   = 0.01
tend = 1.0

u_old = GridFunction(V)
u_old.vec.data = velocity.vec

# Implicit-Diffusion + Mass (für Newton-Schritt)
mstar = BilinearForm(X)
mstar += InnerProduct(u, v)*dx + dt*nu*InnerProduct(grad(u), grad(v))*dx
mstar += -dt*(div(u)*q + div(v)*p)*dx
mstar.Assemble()
inv_mstar = mstar.mat.Inverse(freedofs=X.FreeDofs(), inverse="umfpack")

t = 0.0
while t < tend:
    print("t =", round(t,3))

    # Konvektion mit eingefrorenem u_old
    conv = BilinearForm(X, nonassemble=True)
    conv += InnerProduct(grad(u)*u_old, v)*dx
    conv.Assemble()

    # Residuum bilden
    res = (conv.mat + stokes.mat) * gfu.vec

    # Update
    gfu.vec.data -= dt * inv_mstar * res

    # Randwerte nach dem Schritt wieder setzen
    velocity.Set(uin, definedon=mesh.Boundaries("inlet"))
    velocity.Set((0,0), definedon=mesh.Boundaries("wall|circle"))

    Draw(velocity, mesh, "ns-velocity")
    t += dt


TypeError: Rectangle(): incompatible function arguments. The following argument types are supported:
    1. (self: netgen.libngpy._NgOCC.WorkPlane, l: typing.SupportsFloat, w: typing.SupportsFloat, name: str | None = None) -> netgen.libngpy._NgOCC.WorkPlane

Invoked with: <netgen.libngpy._NgOCC.WorkPlane object at 0x0000028D417D1F70>, <netgen.libngpy._meshing.Point2d object at 0x0000028D42F78AB0>, <netgen.libngpy._meshing.Point2d object at 0x0000028D42DBDB30>

In [ ]:
# # navierstokes_channel_ngsolve.py

# from ngsolve import *
# from ngsolve.webgui import *     # WICHTIG für Notebook/WebGUI
# from netgen.geom2d import *

# # ---------------------------------------------------------
# # 1) Geometrie und Mesh
# # ---------------------------------------------------------
# ngmesh = unit_square.GenerateMesh(maxh=0.07)
# mesh   = Mesh(ngmesh)

# # Boundaries des unit_square: "left", "right", "bottom", "top"
# inlet  = "left"
# walls  = "top|bottom"
# outlet = "right"

# print("Boundaries:", mesh.GetBoundaries())

# # ---------------------------------------------------------
# # 2) Taylor–Hood FE-Räume
# #    V: Geschwindigkeit (Vektor-H1, Ordnung 2)
# #    Q: Druck (H1, Ordnung 1, mit Fixierung des Drucks an 'left')
# # ---------------------------------------------------------
# V = VectorH1(mesh, order=2, dirichlet=inlet + "|" + walls)
# Q = H1(mesh,      order=1, dirichlet=inlet)   # Druck-Fixierung, nur für Eindeutigkeit

# X = V * Q
# (u, p) = X.TrialFunction()
# (v, q) = X.TestFunction()

# gfu    = GridFunction(X)   # (u,p) zusammen
# velocity, pressure = gfu.components
# u_old  = GridFunction(V)   # Geschwindigkeit zum vorherigen Zeitschritt

# # ---------------------------------------------------------
# # 3) Parameter
# # ---------------------------------------------------------
# nu   = 0.01    # kinematische Viskosität (1/Re)
# dt   = 0.01    # Zeitschritt
# tend = 1.0     # Endzeit

# # parabolisches Inflow-Profil: u_x(y) = 4*y*(1-y), u_y = 0
# inflow = CoefficientFunction((4*y*(1-y), 0))

# # Anfangsbedingung: Ruhe im Volumen, Rand gesetzt
# velocity.Set((0,0))  # überall 0
# velocity.Set((0,0), definedon=mesh.Boundaries(walls))
# velocity.Set(inflow, definedon=mesh.Boundaries(inlet))
# u_old.vec.data = velocity.vec

# # ---------------------------------------------------------
# # 4) Visualisierung
# # ---------------------------------------------------------
# Draw(velocity, mesh, "velocity")
# Draw(pressure, mesh, "pressure")

# # ---------------------------------------------------------
# # 5) Zeitschleife (semi-impliziter NS-Schritt)
# #
# # (u^{n+1} - u^n)/dt + (u^n · ∇) u^{n+1} - nu Δu^{n+1} + ∇p^{n+1} = 0
# # div u^{n+1} = 0
# # ---------------------------------------------------------
# t = 0.0
# while t < tend + 1e-10:
#     print(f"t = {t:6.3f}")

#     # Bilinearform A(u^{n+1},p^{n+1})
#     a = BilinearForm(X, symmetric=False)

#     # Zeitterm und Diffusion
#     a += (1/dt) * InnerProduct(u, v) * dx
#     a += nu     * InnerProduct(grad(u), grad(v)) * dx

#     # Konvektion mit eingefrorener u_old: (u_old · ∇) u^{n+1}
#     a += InnerProduct(grad(u) * u_old, v) * dx

#     # Druckkopplung
#     a += -div(v) * p * dx
#     a += -div(u) * q * dx

#     a.Assemble()

#     # rechte Seite: (1/dt) * u_old
#     f = LinearForm(X)
#     f += (1/dt) * InnerProduct(u_old, v) * dx
#     f.Assemble()

#     # Randwerte in aktueller Lösung festsetzen (inhomogene Dirichlet-Bedingungen)
#     velocity.Set((0,0), definedon=mesh.Boundaries(walls))
#     velocity.Set(inflow, definedon=mesh.Boundaries(inlet))

#     # Lifting-Verfahren für inhomogene Dirichlet-Ränder:
#     # Löse A * delta = f - A * gfu  auf freien Freiheitsgraden
#     res = f.vec - a.mat * gfu.vec
#     inv = a.mat.Inverse(freedofs=X.FreeDofs(), inverse="umfpack")
#     gfu.vec.data += inv * res

#     # u^{n+1} wird zu u^n für den nächsten Schritt
#     u_old.vec.data = velocity.vec

#     Redraw()
#     t += dt


Boundaries: ('bottom', 'right', 'top', 'left')


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

t =  0.000


NgException: SparseMatrix::InverseMatrix:  UmfpackInverse not available

In [ ]:

# # navierstokes_channel.py

# from ngsolve import *
# from ngsolve.webgui import Draw
# from netgen.geom2d import unit_square

# # ---------------------------------------------------------
# # 1) Geometrie und Mesh
# # ---------------------------------------------------------
# ngmesh = unit_square.GenerateMesh(maxh=0.07)
# mesh   = Mesh(ngmesh).Curve(3)

# # Boundaries des unit_square: "left", "right", "top", "bottom"
# # Wir interpretieren:
# #  left   = Inlet  (parabolisches Einströmen)
# #  right  = Outlet ("do nothing")
# #  top/bottom = Wände (no-slip)
# print("Boundaries:", mesh.GetBoundaries())

# # ---------------------------------------------------------
# # 2) Taylor–Hood Räume: Geschwindigkeit / Druck
# # ---------------------------------------------------------
# k = 2                                    # Geschw.-Ordnung
# V = VectorH1(mesh, order=k,
#              dirichlet="left|top|bottom")  # u=Randwert auf Inlet + Wänden
# Q = H1(mesh, order=k-1)                  # Druck ohne Dirichlet
# X = V * Q

# (u, p), (v, q) = X.TnT()


# # globale Gitterfunktion (u,p)
# gfu = GridFunction(X)
# velocity, pressure = gfu.components

# # ---------------------------------------------------------
# # 3) Randwerte für die Anfangs-Stokes-Lösung
# # ---------------------------------------------------------
# nu = 0.01   # kinematische Viskosität

# # parabolisches Inflow-Profil auf der linken Seite: u_x(y) = 4*y*(1-y)
# uin = CoefficientFunction((4*y*(1-y), 0))


# # no-slip an top/bottom, Inflow am left
# velocity.Set((0, 0), definedon=mesh.Boundaries("top|bottom"))
# velocity.Set(uin,    definedon=mesh.Boundaries("left"))
# print("zero4")

# # Visualisierung
# Draw(velocity, mesh, "velocity")
# Draw(pressure, mesh, "pressure")

# # ---------------------------------------------------------
# # 4) Stokes-Problem (für Anfangswerte)
# #     a((u,p),(v,q)) = nu * (∇u:∇v) - (div u) q - (div v) p
# # ---------------------------------------------------------
# stokes_form = ( nu * InnerProduct(grad(u), grad(v))
#                 - div(u) * q
#                 - div(v) * p ) * dx

# print("first")

# a = BilinearForm(X)
# a += stokes_form
# a.Assemble()

# f = LinearForm(X)     # hier keine Volumenkräfte → 0
# f.Assemble()

# # Randwerte sind bereits in gfu gesetzt.
# # Jetzt Stokes-System mit inhomogenen Dirichletbedingungen lösen:
# res = f.vec - a.mat * gfu.vec
# inv_stokes = a.mat.Inverse(freedofs=X.FreeDofs(), inverse="umfpack")
# gfu.vec.data += inv_stokes * res

# Redraw()

# print("second")
# # ---------------------------------------------------------
# # 5) Zeitdiskretisierung (IMEX Euler wie im NGSolve-Tutorial)
# #     m(Δu^{n+1}, v) + dt * a((Δu^{n+1},Δp^{n+1}),(v,q))
# #         = dt * ( -a((u^n,p^n),(v,q)) - c(u^n;u^n,v) )
# # ---------------------------------------------------------
# dt   = 0.001
# tend = 1.0

# # Matrix für impliziten Teil:
# mstar = BilinearForm(X)
# mstar += InnerProduct(u, v) * dx + dt * stokes_form
# mstar.Assemble()
# inv_mstar = mstar.mat.Inverse(freedofs=X.FreeDofs(), inverse="umfpack")

# # Konvektion: c(w;u,v) = ∫ (w · ∇u) · v dx
# # Hier als "semi-lineare" Bilinearform (nonassemble=True):
# conv = BilinearForm(X, nonassemble=True)
# conv += (Grad(u) * u) * v * dx   # w = u, u = u, v = v
#                                   # Grad(u)*u entspricht (u·∇)u

# # ---------------------------------------------------------
# # 6) Zeitschleife
# # ---------------------------------------------------------
# print("third")
# t = 0.0
# while t < tend - 0.5*dt:
#     print("t =", t)

#     # Konvektionsform mit aktuellem Zustand gfu auswerten
#     conv.Assemble()

#     # Residuum: (a + conv) * gfu
#     res = (a.mat + conv.mat) * gfu.vec

#     # Inkrement Δ(u,p) lösen:
#     gfu.vec.data -= dt * inv_mstar * res

#     # Randwerte nach Numerik-Schritt wieder „scharf“ setzen:
#     velocity.Set((0, 0), definedon=mesh.Boundaries("top|bottom"))
#     velocity.Set(uin,    definedon=mesh.Boundaries("left"))

#     Redraw()
#     t += dt


Boundaries: ('bottom', 'right', 'top', 'left')
zero4


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

first


NgException: SparseMatrix::InverseMatrix:  UmfpackInverse not available